# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import geoviews as gv
import geoviews.tile_sources as gvts
import holoviews as hv
import pandas as pd
import requests
import warnings
warnings.filterwarnings('ignore')
# Import API key
from api_keys import geoapify_key

c:\Users\vinay\anaconda3\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mazatlan,23.2167,-106.4167,30.51,82,100,1.34,MX,1697850435
1,1,thompson,55.7435,-97.8558,7.09,100,100,2.57,CA,1697850503
2,2,san antonio de pale,-1.4014,5.6325,25.81,83,99,4.05,GQ,1697850503
3,3,grytviken,-54.2811,-36.5092,2.22,95,100,1.52,GS,1697850503
4,4,happy valley-goose bay,53.3168,-60.3315,7.92,93,100,0.41,CA,1697850503


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display


# Create a scatter plot with hvplot
custom_palette = 'coolwarm'
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    color='Humidity',
    hover_cols=['City'],
    alpha=0.5,
    height=500,
    width=800,
    frame_width=800,
    frame_height=500,
    cmap=custom_palette,
    title='Humidity (%) by Location'
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 18
max_temp = 28
cloudiness_criteria = 0

# Filter cities that meet the criteria
filtered_cities = city_data_df[
    (city_data_df['Max Temp'] > min_temp) &
    (city_data_df['Max Temp'] < max_temp ) &
    (city_data_df['Cloudiness'] == cloudiness_criteria)
]
# Drop rows with null values (if any)
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.head(20)



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,saint-pierre,-21.3393,55.4781,22.82,68,0,3.09,RE,1697850504
46,46,key west,24.5557,-81.7826,26.00,76,0,3.09,US,1697850447
86,86,al kharijah,25.4514,30.5464,20.90,37,0,4.05,EG,1697850517
88,88,ha'il,27.5219,41.6907,21.01,35,0,1.03,SA,1697850518
99,99,harlingen,26.1906,-97.6961,27.25,69,0,3.09,US,1697850519
127,127,grantsville,40.5999,-112.4644,20.20,56,0,1.79,US,1697850526
128,128,beitbridge,-22.2167,30.0000,21.86,43,0,2.04,ZW,1697850526
140,140,siwa oasis,29.2041,25.5195,22.99,32,0,2.25,EG,1697850528
163,163,sur,22.5667,59.5289,26.70,61,0,4.12,OM,1697850534
167,167,guerrero negro,27.9769,-114.0611,22.92,76,0,6.88,MX,1697850534


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''


# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
13,saint-pierre,RE,-21.3393,55.4781,68,
46,key west,US,24.5557,-81.7826,76,
86,al kharijah,EG,25.4514,30.5464,37,
88,ha'il,SA,27.5219,41.6907,35,
99,harlingen,US,26.1906,-97.6961,69,
127,grantsville,US,40.5999,-112.4644,56,
128,beitbridge,ZW,-22.2167,30.0000,43,
140,siwa oasis,EG,29.2041,25.5195,32,
163,sur,OM,22.5667,59.5289,61,
167,guerrero negro,MX,27.9769,-114.0611,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Set the filter parameter to search for hotels
    params = {
        "categories": ["accommodation.hotel"],
        "radius": radius,
        "lat": lat,
        "lon": lng,
        "apiKey": geoapify_key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    try:
        # Make an API request using the params dictionary
        response = requests.get(base_url, params=params)

        # Check the response status code
        response.raise_for_status()
        
        # Convert the API response to JSON format
        data = response.json()

        # Check if hotels were found
        if "features" in data:
            hotels = data["features"]
            if hotels:
                # Grab the first hotel from the results and store the name in the hotel_df DataFrame
                hotel_name = hotels[0]["properties"]["name"]
                hotel_df.loc[index, "Hotel Name"] = hotel_name
            else:
                hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    except requests.exceptions.RequestException as e:
        print(f"Request error: {e}")
        hotel_df.loc[index, "Hotel Name"] = "Request error"

    except Exception as ex:
        print(f"An error occurred: {ex}")
        hotel_df.loc[index, "Hotel Name"] = "Error occurred"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)


Starting hotel search
saint-pierre - nearest hotel: Tropic Hotel
key west - nearest hotel: Harborside Hotel & Marina
al kharijah - nearest hotel: Kharga Hotel
ha'il - nearest hotel: فندق كلاسيك
harlingen - nearest hotel: Holiday Inn Express
grantsville - nearest hotel: Holiday Inn Express
beitbridge - nearest hotel: Beitbridge Inn & Casino
siwa oasis - nearest hotel: فندق الكيلانى
sur - nearest hotel: Sur Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
crane - nearest hotel: Mainstay Suites
san rafael del yuma - nearest hotel: Cadaques Caribe
bam - nearest hotel: هتل جهانگردی بم
diplo - nearest hotel: Noor Muhammad Shoro Hotel
tura - nearest hotel: Eco Park Rest House
alaghsas - nearest hotel: Hôtel Zagado
kamthi - nearest hotel: Shanti Nivas
tazacorte - nearest hotel: App Leyma
alice springs - nearest hotel: Aurora Alice Springs
ar rutbah - nearest hotel: Nuzal Turaif
An error occurred: 'name'
orkney - nearest hotel: Error occurred
matoury - nearest hotel: La Marmotte
tamanras

,City,Country,Lat,Lng,Humidity,Hotel Name
13,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel
46,key west,US,24.5557,-81.7826,76,Harborside Hotel & Marina
86,al kharijah,EG,25.4514,30.5464,37,Kharga Hotel
88,ha'il,SA,27.5219,41.6907,35,فندق كلاسيك
99,harlingen,US,26.1906,-97.6961,69,Holiday Inn Express
127,grantsville,US,40.5999,-112.4644,56,Holiday Inn Express
128,beitbridge,ZW,-22.2167,30.0000,43,Beitbridge Inn & Casino
140,siwa oasis,EG,29.2041,25.5195,32,فندق الكيلانى
163,sur,OM,22.5667,59.5289,61,Sur Hotel
167,guerrero negro,MX,27.9769,-114.0611,76,Plaza sal paraiso


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
custom_palette = 'coolwarm'

hotel_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    color='Humidity',
    hover_cols=['Hotel Name'],
    alpha=0.5,
    height=700,
    width=900,
    cmap=custom_palette,
    title='Hotel Locations'
)


# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name)